<a href="https://colab.research.google.com/github/INVISIBLE-SAM/using-gfpgan-for-video-recovery/blob/main/Video_Frame_Processing_and_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install basicsr==1.4.2
!pip install gfpgan==1.3.8
!pip install torchvision==0.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 25.1 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214818 sha256=21b990c5caa5570cd91966ebc75392e9bb7fef174f2af4fca9c7b0b5a2a88764
  Stored in directory: /root/.cache/pip/wheels/38/83/99/2d8437cc652a01af27df5ff037a4075e95b52d67705c5f30ca
Successfully built basicsr
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.8 MB/s eta 

In [ ]:
import cv2
import os
#download video
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Hyq6aQHQAnTL6qEJhab6ijsW74wyBuOb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Hyq6aQHQAnTL6qEJhab6ijsW74wyBuOb" -O video.mp4 && rm -rf /tmp/cookies.txt

video=cv2.VideoCapture('video.mp4')


--2024-12-08 05:27:20--  https://docs.google.com/uc?export=download&confirm=&id=1Hyq6aQHQAnTL6qEJhab6ijsW74wyBuOb
Resolving docs.google.com (docs.google.com)... 142.251.2.102, 142.251.2.101, 142.251.2.113, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1Hyq6aQHQAnTL6qEJhab6ijsW74wyBuOb&export=download [following]
--2024-12-08 05:27:21--  https://drive.usercontent.google.com/download?id=1Hyq6aQHQAnTL6qEJhab6ijsW74wyBuOb&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33252047 (32M) [video/mp4]
Saving to: ‘video.mp4’

video.mp4           100%[===================>]  31.71M  54.5MB/s    in 0.6s    

2

In [ ]:
#load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#output directory for cropped faces
output_dir ="cropped_faces"
os.makedirs(output_dir,exist_ok=True)

frame_count=0
while True:
  ret,frame=video.read()
  if not ret:
    break

  gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  faces =face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))

  if len(faces)>0: #find largest face
    largest_face =max(faces,key=lambda x:x[2]*x[3])
    x,y,w,h=largest_face

    #add padding
    x-=20
    y-=20
    w+=40
    h+=40

    #ensure coordinates are within frame boundries
    x=max(x,0)
    y=max(y,0)
    w=min(w,frame.shape[1]-x)
    h=min(h,frame.shape[0]-y)

    cropped_faces=frame[y:y+h,x:x+w]

    #resize the image
    resized_face = cv2.resize(cropped_faces,(512,512))

    #save the cropped faces
    output_filename =os.path.join(output_dir,f"face_{len(os.listdir(output_dir))+1}.jpg")
    cv2.imwrite(output_filename,resized_face)
    frame_count +=1

video.release()
print(frame_count)


1125


In [ ]:
from PIL import Image
import torch
from gfpgan import GFPGANer
from torchvision import transforms
import numpy as np

input_dir = '/content/cropped_faces'  # Directory with face images
output_dir = '/content/restored_faces'  # Directory to save restored images
model_path = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth'


os.makedirs(output_dir, exist_ok=True)

restorer=GFPGANer(
    model_path=model_path,
    upscale=2,

    channel_multiplier=2,

    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

#Process images
for filename in os.listdir(input_dir):
  input_path =os.path.join(input_dir,filename)

  #load the input image
  img=Image.open(input_path).convert('RGB')
  img_np=np.array(img)

  #perform restoration
  cropped_faces,restored_img,restored_with_background=restorer.enhance(img_np,has_aligned=False,only_center_face=False,paste_back=True)

  #save the restored image
  output_path = os.path.join(output_dir, filename)
  restored_pil = Image.fromarray(restored_with_background)
  restored_pil.save(output_path)


  print(f"Restored image saved to: {output_path}")

print("Processing complete!")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Restored image saved to: /content/restored_faces/face_800.jpg
Restored image saved to: /content/restored_faces/face_543.jpg
Restored image saved to: /content/restored_faces/face_759.jpg
Restored image saved to: /content/restored_faces/face_896.jpg
Restored image saved to: /content/restored_faces/face_2360.jpg
Restored image saved to: /content/restored_faces/face_2611.jpg
Restored image saved to: /content/restored_faces/face_53.jpg
Restored image saved to: /content/restored_faces/face_46.jpg
Restored image saved to: /content/restored_faces/face_1851.jpg
Restored image saved to: /content/restored_faces/face_809.jpg
Restored image saved to: /content/restored_faces/face_1560.jpg
Restored image saved to: /content/restored_faces/face_801.jpg
Restored image saved to: /content/restored_faces/face_1809.jpg
Restored image saved to: /content/restored_faces/face_571.jpg
Restored image saved to: /content/restored_faces/face_444.jpg
Restored image saved to: /content/restored_faces/face_247.jpg
Resto

In [ ]:

import cv2
import os

def generate_video_from_images(image_dir, output_video_path, fps=30):
    """Generates a video from a sequence of images.

    Args:
        image_dir: The directory containing the images.
        output_video_path: The path to save the output video.
        fps: Frames per second for the output video.
    """

    images = [img for img in os.listdir(image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
    images.sort()

    if not images:
        print("No images found in the directory.")
        return

    # Determine image dimensions from the first image
    first_image_path = os.path.join(image_dir, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Add images to the video
    for image in images:
        image_path = os.path.join(image_dir, image)
        img = cv2.imread(image_path)
        video.write(img)

    # Release the video writer
    cv2.destroyAllWindows()
    video.release()
    print(f"Video generated successfully at: {output_video_path}")



input_image_dir = '/content/restored_faces'
output_video_file = '/content/output_video.mp4'
generate_video_from_images(input_image_dir, output_video_file)

Video generated successfully at: /content/output_video.mp4
